In [1]:
from glotaran_tools.specification_parser import parse_yml
from glotaran_core.model import Dataset
from glotaran_models.kinetic import KineticSeperableModel
import numpy as np
# Imports for plotting

import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
test_parameter = [101e-4, 202e-5, 505e-5, 1]
test_amp = 5.6

In [3]:
class SimpleDecay(Dataset):
    def __init__(self, amp, rate):
        self.timepoints = np.asarray(np.arange(0, 200, 1.5)).tolist()
        self._channel = []

        for i in range(len(self.timepoints)):
            self._channel.append(
                amp * np.exp(-self.timepoints[i] * 10 * rate)
            )

    def channels(self):
        return [self._channel]
    
dataset = SimpleDecay(test_amp, test_parameter[0])
# plot 2 channels
# fig1 = plt.figure()
#plt.plot(dataset.channels()[0])

In [18]:
spec = '''
type: kinetic

parameter: {}

initial_concentrations:
    - label: i1
      parameter: [4,4,4]

megacomplexes:
    - label: mc1
      k_matrices: [k1]

k_matrices:
  - label: "k1"
    matrix: {{
      '(1,1)': 1,
      '(2,2)': 2,
      '(3,3)': 3,
}}

irf: []

datasets: 
  - label: dataset1
    type: spectral
    megacomplexes: [mc1]
    path: 'data/dataset1.ascii'

'''.format(test_parameter)
           
model = parse_yml(spec)
print(model.k_matrices['k1'].matrix.toarray().astype(float))

[[ 1.  0.  0.]
 [ 0.  2.  0.]
 [ 0.  0.  3.]]


In [19]:
fitmodel = KineticSeperableModel(model)
print(fitmodel.get_initial_fitting_parameter())
print(fitmodel._construct_k_matrix(fitmodel.get_initial_fitting_parameter(), model.k_matrices['k1'].matrix.toarray().astype(float)))
print(fitmodel._construct_k_matrix_eigen(fitmodel.get_initial_fitting_parameter(), 
                                                 model.k_matrices['k1'].matrix.toarray().astype(float)))

Parameters([('p1', <Parameter 'p1', 0.0101, bounds=[-inf:inf]>), ('p2', <Parameter 'p2', 0.00202, bounds=[-inf:inf]>), ('p3', <Parameter 'p3', 0.00505, bounds=[-inf:inf]>), ('p4', <Parameter 'p4', 1.0, bounds=[-inf:inf]>)])
[[-0.0101   0.       0.     ]
 [ 0.      -0.00202  0.     ]
 [ 0.       0.      -0.00505]]
(array([-0.00202, -0.00505, -0.0101 ]), array([[ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.]]))


In [20]:
print(model.datasets['dataset1'])

Dataset 'dataset1'

	Dataset Scaling: None
	Initial Concentration: []
	Megacomplexes: ['mc1']

	Irf: None


In [21]:
print(model.k_matrices)

{'k1': <glotaran_models.kinetic.k_matrix.KMatrix object at 0x7fd8f15c32b0>}


In [22]:
f = np.vectorize(
            lambda i: 7 if i != 0 else 5)

In [23]:
print(f(model.k_matrices['k1'].matrix.toarray()))

[[7 5 5]
 [5 7 5]
 [5 5 7]]


In [24]:
#Usage:
#  addAtPos(xycoor)
#    - mat1  : matrix to be added
#    - mat2  : add this matrix to mat1
#    - xycoor: tuple (x,y) containing coordinates
def addAtPos(mat1, mat2, xycoor):
    size_x, size_y = np.shape(mat2)
    coor_x, coor_y = xycoor
    end_x, end_y   = (coor_x + size_x), (coor_y + size_y)
    mat1[coor_x:end_x, coor_y:end_y] = mat1[coor_x:end_x, coor_y:end_y] + mat2
    return mat1

In [25]:
A = np.array([[1,1],[1,1]])
B = np.asarray([[2,2,2],[2,2,2], [2,2,2]])
print(B.shape)
print(addAtPos(B,A,(0,0)))


(3, 3)
[[3 3 2]
 [3 3 2]
 [2 2 2]]


In [26]:
fitmodel.c_matrix(fitmodel.get_initial_fitting_parameter().valuesdict(), *dataset.timepoints)

array([[ 1.        ,  1.        ,  1.        ],
       [ 1.0030346 ,  1.00760376,  1.01526534],
       [ 1.0060784 ,  1.01526534,  1.03076372],
       [ 1.00913144,  1.02298518,  1.04649868],
       [ 1.01219374,  1.03076372,  1.06247384],
       [ 1.01526534,  1.0386014 ,  1.07869287],
       [ 1.01834626,  1.04649868,  1.09515948],
       [ 1.02143653,  1.05445601,  1.11187747],
       [ 1.02453618,  1.06247384,  1.12885066],
       [ 1.02764523,  1.07055264,  1.14608295],
       [ 1.03076372,  1.07869287,  1.1635783 ],
       [ 1.03389167,  1.08689499,  1.18134072],
       [ 1.03702911,  1.09515948,  1.1993743 ],
       [ 1.04017607,  1.10348682,  1.21768316],
       [ 1.04333259,  1.11187747,  1.23627151],
       [ 1.04649868,  1.12033192,  1.25514361],
       [ 1.04967438,  1.12885066,  1.27430381],
       [ 1.05285971,  1.13743417,  1.2937565 ],
       [ 1.05605472,  1.14608295,  1.31350613],
       [ 1.05925942,  1.1547975 ,  1.33355726],
       [ 1.06247384,  1.1635783 ,  1.353

In [27]:
result = fitmodel.fit(fitmodel.get_initial_fitting_parameter(), 
                      *dataset.timepoints, 
                      **{"data": np.asarray(dataset.channels()).T}
                     )
print(result.params)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         3.1925e+01                                    5.88e+02    
       1              2         4.0839e+00      2.78e+01       1.00e+00       3.93e+01    
       2              5         1.9604e-03      4.08e+00       1.25e-01       2.27e+00    
       3              6         4.5970e-07      1.96e-03       2.50e-01       3.56e-02    
       4              7         2.3468e-14      4.60e-07       5.00e-01       8.05e-06    
       5              8         1.5616e-28      2.35e-14       1.00e+00       6.25e-13    
`gtol` termination condition is satisfied.
Function evaluations 8, initial cost 3.1925e+01, final cost 1.5616e-28, first-order optimality 6.25e-13.
Parameters([('p1', <Parameter 'p1', -0.1010000000000004, bounds=[-inf:inf]>), ('p2', <Parameter 'p2', -0.57900967509172319, bounds=[-inf:inf]>), ('p3', <Parameter 'p3', 2.0429931570451454, bounds=[-inf:inf]>), ('p4'

In [28]:
print(np.asarray(dataset.channels()).T.shape)

(134, 1)
